<a href="https://colab.research.google.com/github/surya-moorthy/Mlops/blob/main/cifarImageClassifier_py.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
from torch import nn, save
import torchvision
from torchvision.transforms import transforms,ToTensor
from torchvision.datasets import CIFAR10
from torch.utils.data import DataLoader

device =  torch.device("cuda" if torch.cuda.is_available() else "cpu")

dataset_train = CIFAR10(root="data",download=True,train=True,transform=ToTensor())
dataset = DataLoader(dataset_train,batch_size=32,shuffle=True)

targets = dataset_train.targets
num_of_classes = len(set(targets))

images, labels = next(iter(dataset))

print(images.shape)
print(images[0].shape)

class ClassImagesCifar(nn.Module):
    def __init__(self, num_classes):
        super(ClassImagesCifar,self).__init__()
        self.conv_layer1 = nn.Conv2d(in_channels=3,out_channels=64,kernel_size=2,padding=1)
        self.relu1 = nn.ReLU()
        self.maxpool1 = nn.MaxPool2d(kernel_size=2,stride=2,padding=1)
        self.conv_layer2 = nn.Conv2d(in_channels=64,out_channels=128,kernel_size=2,padding=1)
        self.relu2 = nn.ReLU()
        self.maxpool2 = nn.MaxPool2d(kernel_size=2,stride=2,padding=1)
        self.conv_layer3 = nn.Conv2d(in_channels=128,out_channels=256,kernel_size=2,padding=1)
        self.relu3 = nn.ReLU()
        self.maxpool3 = nn.MaxPool2d(kernel_size=2,stride=2,padding=1)
        self.conv_layer4 = nn.Conv2d(in_channels=256,out_channels=256,kernel_size=2,padding=1)
        self.relu4 = nn.ReLU()
        self.maxpool4 = nn.MaxPool2d(kernel_size=2,stride=2,padding=1)
        self.flatten = nn.Flatten()

        self.fc6 = nn.Linear(4096,512)
        self.relu6 = nn.ReLU()
        self.dropout7 = nn.Dropout(0.5)
        self.fc7 = nn.Linear(512,256)
        self.relu7 = nn.ReLU()
        self.fc8 = nn.Linear(256,num_classes)

    def forward(self,x):
        out = self.conv_layer1(x)
        out = self.relu1(out)
        out = self.maxpool1(out)

        out = self.conv_layer2(out)
        out = self.relu2(out)
        out = self.maxpool2(out)

        out = self.conv_layer3(out)
        out = self.relu3(out)
        out = self.maxpool3(out)

        out = self.conv_layer4(out)
        out = self.relu4(out)
        out = self.maxpool4(out)

        out = self.flatten(out)
        out = self.fc6(out)
        out = self.relu6(out)

        out = self.dropout7(out)
        out = self.fc7(out)
        out = self.relu7(out)

        out = self.fc8(out)
        return out

clf = ClassImagesCifar(num_of_classes).to(device)
loss_func = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(clf.parameters(),lr=0.0001)

if __name__ == "__main__":
    for epoch in range(25):
        for batch in dataset:
            X, y = batch
            X = X.to(device)
            y = y.to(device)

            output = clf(X)
            loss = loss_func(output,y)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
        print(f"Epoch {epoch} has a loss of {loss}")

with open('model_compiled.pt',"wb") as f:
    save(clf.state_dict(),f)



100%|██████████| 170M/170M [00:06<00:00, 28.1MB/s]


Extracting data/cifar-10-python.tar.gz to data
torch.Size([32, 3, 32, 32])
torch.Size([3, 32, 32])
Epoch 0 has a loss of 2.294776201248169
Epoch 1 has a loss of 2.28483247756958
Epoch 2 has a loss of 2.2948086261749268
Epoch 3 has a loss of 2.305663585662842
Epoch 4 has a loss of 2.3033711910247803
Epoch 5 has a loss of 2.302550792694092
Epoch 6 has a loss of 2.3014731407165527
Epoch 7 has a loss of 2.3094286918640137
Epoch 8 has a loss of 2.3060662746429443
Epoch 9 has a loss of 2.293626308441162
Epoch 10 has a loss of 2.3046398162841797
Epoch 11 has a loss of 2.311518669128418
Epoch 12 has a loss of 2.2993099689483643
Epoch 13 has a loss of 2.303626298904419
Epoch 14 has a loss of 2.298217296600342
Epoch 15 has a loss of 2.3078815937042236
Epoch 16 has a loss of 2.3015100955963135
Epoch 17 has a loss of 2.300851345062256
Epoch 18 has a loss of 2.3105242252349854
Epoch 19 has a loss of 2.3006255626678467
Epoch 20 has a loss of 2.306800365447998
Epoch 21 has a loss of 2.293592214584350

In [ ]:
from torch import load
from PIL import Image
print(targets)
with open('model_compiled.pt',"rb") as f:
        clf.load_state_dict(load(f))
img = Image.open('bird6.png')
img_tensor = ToTensor()(img).unsqueeze(0).to('cuda')
print(torch.argmax(clf(img_tensor)))

tensor(9, device='cuda:0')


<ipython-input-16-ec8d690f2337>:5: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  clf.load_state_dict(load(f))
